In [20]:
!pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [21]:
from z3 import *
from itertools import combinations

In [22]:
instances_path = "../../instances/"
with open(instances_path + "ins-10.txt") as f:
    lines = [f[:-1] for f in f.readlines()]
    w, n = int(lines[0]), int(lines[1])
    b = [d.split() for d in lines[2:]]
    b = [(int(bw), int(bh)) for (bw, bh) in b]

In [23]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), And(at_most_one_np(bool_vars, name)))

In [24]:
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars, name=""):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name)) 

In [25]:
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name=""):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

### Parameters
- $n$ number of blocks
- $B_i$, $i = 0, ..., n-1$ blocks to insert where $B_i = \{x_i, y_i\}$ are the dimensions of the $i$-block, weight and height respectively
- width $w$
- ```rot = True``` if rotation is allowed 

### Constraints:
- The blocks in the grid can't overlap (exactly one value for each cell). $\sum_{k=0}^{n}(\texttt{cells}_{i,j,k}) \leq 1 \land \sum_{k=0}^{n}(\texttt{cells}_{i,j,k}) \geq 1$ $\forall i,j=0,...,n-1 $
- The cell $\texttt{cells}_{i, j}$ is the bottom left border of one block and its surrounding cells must have the same $k$ value. Moreover, inserting a block mustn't enlarge the width $w$. $\texttt{exactly_one} \Big[ \huge{\land_{\small r=i}^{\small i+y_k} \land_{\small c=j}^{\small j+x_k}}$ $\Big( \texttt{cells}_{r,c,k} \land (j+x_k<w)  \land (i+y_k<h) \Big)$ $\forall i,j=0,...,n-1 \Big]$ $\forall k=0,...,n$

In [26]:
def vlsi_sat(n, b, w, rot=False):
    # - Each cell assumes a value between 0 and n (included), where the 0-value
    #        means that no block is inserted in that cell
    # - the height of the grid is the sum of the height of the blocks
    
    #b = [(1, 1)] + b
    h = sum([bh for (bw, bh) in b])

    # cells[i][j] : cell of row i and column j
    cells = [[[Bool(f"cells_{i}_{j}_{k}") for k in range(n + 1)] for j in range(w)] 
         for i in range(h)]
    is_in_block = [[Bool(f"b_{i}_{j}") for j in range(w)] for i in range(h)]
    
    s = Solver()
    # Each cell must contain a value only, corresponding to the block 
    # number e.g. the blocks can't overlap.
    for i in range(h):
        for j in range(w):
            s.add(exactly_one_np([cells[i][j][k] for k in range(n + 1)]))

    # Each block must occur exactly once
    cnst_tmp = []
    for k in range(n):
        (x, y) = b[k]
        cnst = []
        for i in range(h):
            for j in range(w):
                if j + x < w and i + y < h:
                    block = [cells[r][c][k] 
                             for r in range(i, i + y) for c in range(j, j + x)]
                    cnst.append(And(block))
                    
                    cnst_tmp.append(Implies(And(block), And([is_in_block[r][c] == True
                      for r in range(i, i + y) for c in range(j, j + x)])))
                    
                else:
                    cnst.append(False)
        s.add(exactly_one_np([cc for cc in cnst]))
    s.add(cnst_tmp)
    #o = Optimize()
    
    # Check if it's SAT
    s.check()
    if s.check() == sat:
        m = s.model()
        return [(i, j, k) for i in range(h) for j in range(w) 
                for k in range(n + 1) if m.evaluate(cells[i][j][k])]
    return "UNSAT"

In [ ]:
%%time
sol = vlsi_sat(n, b, w)

In [ ]:
sol

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

dim = (76, w)
M = []
for i in range(dim[0]):
    M.append([])
    for j in range(dim[1]):
        M[i].append(sol[i * w + j][2])

ax.matshow(M, cmap=plt.get_cmap('plasma'))